In [10]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = 'Arial'
import os
import operator
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import LabelEncoder
import time
from scipy.interpolate import interp1d
from scipy.io import loadmat
import random



In [2]:
import tensorflow.keras as keras
import tensorflow as tf

In [14]:
from sklearn.model_selection import train_test_split
import sklearn
df1 = pd.read_csv(r'final_dataset.CSV',header = None)
data = df1.values
random.shuffle(data)
x=data[:,0:1000]
y=data[:,1000:1001]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)
x_train, x_valid , y_train , y_valid = train_test_split(x_train,y_train,test_size=0.3)
nb_classes = len(np.unique(np.concatenate((y_train, y_test), axis=0)))
# print(nb_classes)
# print(y.shape)
# print(y)
# print(x.shape)

# transform the labels from integers to one hot vectors
enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
enc.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
y_test = enc.transform(y_test.reshape(-1, 1)).toarray()
y_valid = enc.transform(y_valid.reshape(-1, 1)).toarray()

# save orignal y because later we will use binary

y_true = np.argmax(y_test, axis=1)

if len(x_train.shape) == 2:  # if univariate
    # add a dimension to make it multivariate with one dimension 
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
    x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
    x_valid = x_valid.reshape((x_valid.shape[0], x_test.shape[1], 1))

input_shape = x_train.shape[1:]  

In [18]:
# input_size = (15483,1001)
# inputs = keras.layers.Input(input_size)
n_feature_maps = 64

inputs = keras.layers.Input(input_shape)
print(inputs.shape)

# BLOCK 1

conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(inputs)
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)
print(conv_x.shape)

conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=5, padding='same')(conv_x)
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)
print(conv_y.shape)

conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
conv_z = keras.layers.BatchNormalization()(conv_z)
print(conv_z.shape)

# expand channels for the sum
shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(inputs)
shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
print(shortcut_y.shape)

output_block_1 = keras.layers.add([shortcut_y, conv_z])
output_block_1 = keras.layers.Activation('relu')(output_block_1)
print(output_block_1.shape)

# BLOCK 2

# conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_1)
# conv_x = keras.layers.BatchNormalization()(conv_x)
# conv_x = keras.layers.Activation('relu')(conv_x)
# print(conv_x.shape)

# conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
# conv_y = keras.layers.BatchNormalization()(conv_y)
# conv_y = keras.layers.Activation('relu')(conv_y)
# print(conv_y.shape)

# conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
# conv_z = keras.layers.BatchNormalization()(conv_z)
# print(conv_z.shape)

# # expand channels for the sum
# shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
# shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
# print(shortcut_y.shape)

# output_block_2 = keras.layers.add([shortcut_y, conv_z])
# output_block_2 = keras.layers.Activation('relu')(output_block_2)
# print(output_block_2.shape)

# BLOCK 3

conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_1)
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)

conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)

conv_z = keras.layers.Conv1D(filters=n_feature_maps , kernel_size=3, padding='same')(conv_y)
conv_z = keras.layers.BatchNormalization()(conv_z)

# no need to expand channels because they are equal
shortcut_y = keras.layers.BatchNormalization()(output_block_1)


output_block_3 = keras.layers.add([shortcut_y, conv_z])
output_block_3 = keras.layers.Activation('relu')(output_block_3)
print(output_block_3.shape)

# FINAL

gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)

output_layer1 = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)
output_layer = keras.layers.Dense(nb_classes, activation='softmax')(output_layer1)

print(output_layer.shape)

model = keras.models.Model(inputs=inputs, outputs=output_layer)

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.summary()

(None, 1000, 1)
(None, 1000, 64)
(None, 1000, 64)
(None, 1000, 64)
(None, 1000, 64)
(None, 1000, 64)
(None, 1000, 64)
(None, 6)
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 1000, 1)]    0           []                               
                                                                                                  
 conv1d_32 (Conv1D)             (None, 1000, 64)     576         ['input_5[0][0]']                
                                                                                                  
 batch_normalization_36 (BatchN  (None, 1000, 64)    256         ['conv1d_32[0][0]']              
 ormalization)                                                                                    
                                                               

In [6]:
'''# input_size = (15483,1001)
# inputs = keras.layers.Input(input_size)
n_feature_maps = 64

inputs = keras.layers.Input(input_shape)
print(inputs.shape)

# BLOCK 1

conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(inputs)
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)
print(conv_x.shape)

conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=5, padding='same')(conv_x)
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)
print(conv_y.shape)

conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
conv_z = keras.layers.BatchNormalization()(conv_z)
print(conv_z.shape)

# expand channels for the sum
shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(inputs)
shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
print(shortcut_y.shape)

output_block_1 = keras.layers.add([shortcut_y, conv_z])
output_block_1 = keras.layers.Activation('relu')(output_block_1)
print(output_block_1.shape)

# BLOCK 2

conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_1)
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)
print(conv_x.shape)

conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)
print(conv_y.shape)

conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
conv_z = keras.layers.BatchNormalization()(conv_z)
print(conv_z.shape)

# expand channels for the sum
shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
print(shortcut_y.shape)

output_block_2 = keras.layers.add([shortcut_y, conv_z])
output_block_2 = keras.layers.Activation('relu')(output_block_2)
print(output_block_2.shape)

# BLOCK 3

conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_2)
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)

conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)

conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
conv_z = keras.layers.BatchNormalization()(conv_z)

# no need to expand channels because they are equal
shortcut_y = keras.layers.BatchNormalization()(output_block_2)

output_block_3 = keras.layers.add([shortcut_y, conv_z])
output_block_3 = keras.layers.Activation('relu')(output_block_3)
print(output_block_3.shape)

# FINAL

gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)

output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)
print(output_layer.shape)

model = keras.models.Model(inputs=inputs, outputs=output_layer)

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.summary()'''

(None, 1000, 1)
(None, 1000, 64)
(None, 1000, 64)
(None, 1000, 64)
(None, 1000, 64)
(None, 1000, 64)
(None, 1000, 128)
(None, 1000, 128)
(None, 1000, 128)
(None, 1000, 128)
(None, 1000, 128)
(None, 1000, 128)
(None, 6)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1000, 1)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 1000, 64)     576         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 1000, 64)    256         ['conv1d[0][0]']                 
 alization)                                                              

In [13]:



model.save('model1.h5')

In [20]:

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, min_lr=0.0001)

file_path =  'best_model.hdf5'

model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
                                                    save_best_only=True)

callbacks = [reduce_lr, model_checkpoint]


In [21]:
batch_size = 64
nb_epochs = 10
mini_batch_size = int(min(x_train.shape[0] / 10, batch_size))

hist = model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=nb_epochs,
                        verbose=1, callbacks=callbacks)
model.evaluate(x_test,y_test)

Epoch 1/10
97/97 [==============================] - 151s 2s/step - loss: 1.5851 - accuracy: 0.3778 - lr: 0.0010
Epoch 2/10
97/97 [==============================] - 134s 1s/step - loss: 1.4580 - accuracy: 0.4862 - lr: 0.0010
Epoch 3/10
97/97 [==============================] - 134s 1s/step - loss: 1.3253 - accuracy: 0.5542 - lr: 0.0010
Epoch 4/10
97/97 [==============================] - 132s 1s/step - loss: 1.2630 - accuracy: 0.5882 - lr: 0.0010
Epoch 5/10
97/97 [==============================] - 133s 1s/step - loss: 1.2062 - accuracy: 0.6075 - lr: 0.0010
Epoch 6/10
97/97 [==============================] - 124s 1s/step - loss: 1.1505 - accuracy: 0.6171 - lr: 0.0010
Epoch 7/10
97/97 [==============================] - 121s 1s/step - loss: 1.0993 - accuracy: 0.6246 - lr: 0.0010
Epoch 8/10
97/97 [==============================] - 122s 1s/step - loss: 1.0520 - accuracy: 0.6462 - lr: 0.0010
Epoch 9/10
97/97 [==============================] - 122s 1s/step - loss: 1.0007 - accuracy: 0.6618 - lr:

[1.978264331817627, 0.30646631121635437]

In [9]:
model.save('final_model')
'''
batch_size = 64
nb_epochs = 50
mini_batch_size = int(min(x_train.shape[0] / 10, batch_size))

hist = model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=nb_epochs,
                        verbose=1, callbacks=callbacks)'''
model.evaluate(x_test,y_test)

INFO:tensorflow:Assets written to: final_model\assets
69/69 [==============================] - 17s 246ms/step - loss: 6.2329 - accuracy: 0.2131


[6.232907772064209, 0.21311475336551666]